In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb as qu
import quimb.tensor as qtn
import cotengra as ctg
import quf
import numpy as np
import torch
import algo_cooling as algo
import itertools
import algo_bp
import gen_loop as lg
import gen_loop_tn as lg_tn


In [2]:
opt = algo.opt_(progbar=True)

In [3]:
import torch
to_backend = algo.backend_torch(device = "cuda", dtype = torch.complex128, requires_grad=False)
to_backend_ = algo.backend_numpy( dtype = "complex128")

opt = algo.opt_(progbar=False)

In [4]:
cor = [(1,1),(2,1)]

res_cor = algo_bp.bp_info_rho(cor)
pass_rho = res_cor


obs = qu.pauli("Z") & qu.pauli("Z") 
obs_tensor = qtn.Tensor(obs.reshape(2,2,2,2), inds=res_cor["inds_rho"], tags="zz")
obs_tensor.apply_to_arrays(to_backend)
res_cor 

{'inds_rho': ['k1,1', 'k2,1', 'b1,1', 'b2,1'],
 'reg_reindex': {'k1,1': 'b1,1', 'k2,1': 'b2,1'},
 'reg_tags': ['I1,1', 'I2,1'],
 'leftinds_rho': ['k1,1', 'k2,1'],
 'rightinds_rho': ['b1,1', 'b2,1']}

In [5]:
# Define <peps | peps> as an easy problem for bp convergence:
Lx, Ly, bnd = (3, 3, 3)
peps = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=bnd, seed=12, dtype="complex128")
peps.apply_to_arrays(to_backend)
fix = { f"I{i},{j}":(i,j) for i,j in itertools.product(range(peps.Lx), range(peps.Ly)) }

res_cor["fix"] = fix

In [6]:
tn = peps.H & peps
norm = tn.contract(all, optimize=opt)
peps /= (norm**0.5)
tn = peps.H & peps
norm = tn.contract(all, optimize=opt)
complex(norm)

(1.0000000000000004+4.163336342344337e-17j)

In [7]:
peps.add_tag('KET')
pepsH = peps.conj().retag({'KET': 'BRA'})
tn = pepsH & peps


In [8]:
peps_ = qtn.tensor_network_gate_inds(peps, to_backend(obs.reshape(2,2,2,2)), [res_cor['leftinds_rho'][0], res_cor["leftinds_rho"][1]], contract=False,  inplace=False)

z_tn = pepsH & peps_
zz_exact = z_tn.contract(all, optimize=opt)
print(zz_exact)

tensor(-0.0697+4.3368e-17j, device='cuda:0', dtype=torch.complex128)


In [9]:
bp = algo_bp.L1BP(tn, site_tags=peps.site_tags, damping=0.0, update="parallel", optimize=opt, local_convergence=True)
bp.run(progbar=True, tol = 1.e-11, max_iterations=4000, diis=True)

mantissa, norm_exponent = bp.contract(strip_exponent=True)
est_norm = complex(mantissa * 10**norm_exponent)
print(est_norm)

max|dM|=3.92e-12 nconv: 3/3 : : 76it [00:01, 62.41it/s] 

(1.060537104549958-1.6781266969058446e-18j)


In [10]:
res = algo_bp.run_bp(tn, peps.site_tags, normalize=True, print_=True, opt=opt,
                    progbar=True, tol=1.e-9, max_iterations=800, local_convergence=True,
                    update='parallel', damping=0.00, diis=True)

bp = res["bp"]
tn = res["tn"]
print("bp: zeroth", res["norm"], tn.exponent)


bp runs with the parallel update ~ iter = 800


max|dM|=6.37e-10 nconv: 2/2 : : 46it [00:00, 77.37it/s] 

messages normalized and projects are being cal
tn is being normalized w.r.t bp massages
bp: zeroth tensor(1.0605+2.2099e-17j, device='cuda:0', dtype=torch.complex128) tensor(0.0255-1.9148e-17j, device='cuda:0', dtype=torch.complex128)


In [11]:
#all edges in tn:
edges = [tuple(sorted(i)) for i in bp.edges.keys()]
max_edges = len(edges)
print("number of edges:", max_edges)


number of edges: 12


In [12]:
#tags_cluster = [ f"I{i},{j}" for i in range(0,3) for j in range(0,3)]
tags_cluster = []
#tags_cluster = None
chi = 100
Z_f = []
obs_f = []
norm_f = []
loop_f = []
number_edges_f = []
obs_error_f = []
norm_error_f = []
norm_loop_f = []

for i in range(0, max_edges+1):
    loops = lg.loop_gen_local(bp, circum = i, tags_cluster=tags_cluster, sites=res_cor["reg_tags"], 
                               circum_g=0, tn_flat=peps, site_tags=peps.site_tags)

    Z, norm_ = lg_tn.rho_bp_excited_loops_(tn, bp, pass_rho, loops, obs_tensor, opt=opt, draw_=False,
                                         pari_exclusive=res_cor["reg_tags"], simplify=False)
    Z_f.append(Z)
    norm_f.append(norm_)


    print("loop", i, len(loops), complex(norm_), complex(Z), "norm", complex(sum(norm_f)),"z", complex(sum(Z_f)/sum(norm_f)), complex(zz_exact))
    print("\n")    
    

loop 0 1 (1.0605371045499563-1.7548430529740666e-17j) (0.00041746918329291227+2.831404940586948e-20j) norm (1.0605371045499563-1.7548430529740666e-17j) z (0.00039363939413517+3.3211287767782764e-20j) (-0.06971473473173234+4.336808689942018e-17j)


loop 1 1 (9.069351172841245e-17-3.819116338160731e-18j) (0.0010228983349428468+2.5763672562660006e-18j) norm (1.0605371045499563-2.13675468679014e-17j) z (0.0013581491039363356+2.483365842651447e-18j) (-0.06971473473173234+4.336808689942018e-17j)


loop 2 0 0j 0j norm (1.0605371045499563-2.13675468679014e-17j) z (0.0013581491039363356+2.483365842651447e-18j) (-0.06971473473173234+4.336808689942018e-17j)


loop 3 2 (-1.734723475976807e-18+1.4256590529598507e-18j) (-0.030900729900144645+2.7588664356942513e-18j) norm (1.0605371045499563-1.9941887814941548e-17j) z (-0.027778719156092632+4.535048910291096e-18j) (-0.06971473473173234+4.336808689942018e-17j)


loop 4 4 (-0.019070445452360196+1.631596930802878e-17j) (-0.021650928888292037+9.240642622